In [22]:
# Standard libraries
import sys
import math
import random
import time
import os

# Third-party imports
import numpy as np
import pandas as pd
import pickle as pkl

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import csv

import importlib
from yapf.yapflib.yapf_api import FormatCode

# Project imports
import ephesus
import utils
from const import *

In [24]:
# Verbosity
# 0 = off
# 1 is ending
# 2 is phases
# 3 is percent injection
verbo = 1

### GPU Setup

In [15]:
# Specific GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[3:4], 'GPU')
mirro_strat = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


### Predictions

In [16]:
# Mask the data
masking = layers.Masking(mask_value=0., input_shape = (1900, 1))

model_file = f'combi-padde_cuts-1255247-1644454393.pkl-0-1645790147.h5'
model = tf.keras.models.load_model(f'{xom_data_path}plane_moon_model/{model_file}');

# Create a prediction csv filename
predi_log_name = f'{model_file}-{int(time.time())}.csv'

In [26]:
for curre_secto in range(1, 27):
    
    predi_data = []
    # Data Preprocessing
    # Find the most recent file in injec_curve
    datas = f'forma_curve-{curre_secto}.pkl'
    # CHANGE TO JUST UNDERSCORE

    with open(f'{xom_data_path}forma_raw_curve/{datas}','rb') as f:
        raw_x_data = pkl.load(f)
        
    # Normalize the data from 0 to 1
    x_data = np.copy(raw_x_data[:, :-1, 1].astype(float))

    for i in range(len(x_data)):
        chang_slots = np.where(x_data[i] != 0)[0]
        x_data[i, chang_slots] = utils.norma_data(x_data[i, chang_slots])

    x_data = np.expand_dims(x_data, 2)

    # Make predictions
    predi = model.predict(x_data).squeeze()

    for i in range(len(raw_x_data)):
        predi_data.append([predi[i], raw_x_data[i, -1, 1]['tic_id'], raw_x_data[i, -1, 1]['cut_numbe'], \
raw_x_data[i, -1, 1]['toi'], raw_x_data[i, -1, 1]['file_name']])
        
    with open(f'{xom_data_path}predi_logs/{predi_log_name}','a') as f:
        write = csv.writer(f)
        write.writerows(predi_data)
        
    if verbo >= 1:
        utils.send_task_comple_email(f'Predicting Complete -- Sector {curre_secto}') 

In [38]:
# Order by index and resave csv
sorte_csv = pd.read_csv(f'{xom_data_path}predi_logs/{predi_log_name}', 
                 names=['predi', 'tic_id', 'cut_numbe', 'toi', 'file_name'])

sorte_csv.sort_values('predi', ascending=False, inplace=True)
sorte_csv.set_index('predi', inplace=True)
sorte_csv.to_csv(f'{xom_data_path}predi_logs/{predi_log_name}')

#### Formatting